In [11]:
import pandas as pd
import folium
import os
import re
from dotenv import load_dotenv

In [12]:
#Allows and calls Environment Variables.
load_dotenv('/Users/marcomayorga/Documents/Code/Pyprojects/CovidTracker/envVariables.env')
apikey = os.getenv("API_TOKEN")

In [13]:
#Makes a call to the api
upToDateCovidData = f'https://api.covidactnow.org/v2/states.json?apiKey={apikey}'

In [14]:
#Makes Pandas read json data from API
sortedCovidData = pd.read_json(upToDateCovidData)
pd.set_option('display.max_column', 17)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 54)

In [15]:
#Scrubs sortedCovidData of unnecisarry data
sortedCovidData = sortedCovidData.drop(['fips', 'county', 'level', 'lat', 'locationId', 'long', 'annotations', 
'metrics', 'riskLevels', 'cdcTransmissionLevel', ], axis=1)
pd.set_option('display.width', 1000)
pd.set_option('display.max_column', 40)
pd.set_option('display.max_rows', 54)

In [16]:
#Loops through all dictionaries in sortedCovidData dataframe containing deaths data in the 'actuals' column 
cases = []
for i in range(len(sortedCovidData)):
    casesKeys = re.split('\s', str(sortedCovidData.loc[i, 'actuals']['cases']))
    casesKeys = re.sub("[\['\]]", "", str(casesKeys))
    cases.append(int(casesKeys))

In [17]:
#will loop through all dictionaries in sortedCovidData dataframe containing deaths data in the 'actuals' column 
deaths = []
for i in range(len(sortedCovidData)):
    deathsKeys = re.split('\s', str(sortedCovidData.loc[i, 'actuals']['deaths']))
    deathsKeys = re.sub("[\['\]]", "", str(deathsKeys))
    deaths.append(int(deathsKeys))

In [18]:
#Make a new Column in the dataframe that will use data extracted from the "actuals" column
sortedCovidData.insert(4, 'cases', cases, True)
sortedCovidData.insert(5, 'deaths', deaths, True)

In [19]:
#Makes items in the state column into strings
newState = []
for i in range(len(sortedCovidData)):
    newState.append(str(sortedCovidData.loc[i, 'state']))
sortedCovidData.insert(1, 'newState', newState, True)
sortedCovidData =sortedCovidData.drop('state', axis=1)

In [20]:
#This Cell Makes the Map
#This pulls the USA State GEODATA from Docs 
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

#Initializes and moves the map
baseMap = folium.Map(location=[48, -102], zoom_start=3)

#Creates cases layer
folium.Choropleth(
    geo_data= state_geo,
    name="Covid Cases",
    data= sortedCovidData,
    columns=["newState", "cases"],
    tooltip=['newState'],
    key_on="feature.id",
    fill_color="BuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Covid Cases",
    show=False
).add_to(baseMap)

#Creates deaths layer
folium.Choropleth(
    geo_data= state_geo,
    name="Covid Deaths",
    data= sortedCovidData,
    columns=["newState", "deaths"],
    key_on="feature.id",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Covid Deaths",
    show=False
).add_to(baseMap)

#Allows user to isolate usa for clearer look at data
folium.map.FeatureGroup("USA", overlay=False).add_to(baseMap)

#Makes the layers clickable
folium.LayerControl(collapsed=False).add_to(baseMap)
baseMap.save("map.html")